In [57]:
!pip install optuna

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
tf.config.list_physical_devices('GPU')
tf.config.experimental.enable_op_determinism()
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score
SEED = 7

In [58]:
# Imprimir los dispositivos físicos disponibles (GPU)
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [59]:
# Cargar el conjunto de datos
datos = pd.read_csv("/content/sample_data/Temp_Asu20092021.csv")

In [60]:
print(type(datos.Fecha))
tipo_dato = datos['Fecha'].dtype
print(tipo_dato)

<class 'pandas.core.series.Series'>
object


In [61]:
# Convertir la columna 'Fecha' al tipo datetime
datos['Fecha'] = pd.to_datetime(datos['Fecha'])
datos

,Unnamed: 0,Fecha,Temperatura,Humedad
0,0,2009-01-01 00:00:00,26.749483,69.609451
1,1,2009-01-01 03:00:00,25.956114,78.225166
2,2,2009-01-01 06:00:00,23.707357,93.551756
3,3,2009-01-01 09:00:00,23.726823,93.394817
4,4,2009-01-01 12:00:00,29.606379,72.155839
...,...,...,...,...
37979,37979,2021-12-31 09:00:00,21.719499,85.092387
37980,37980,2021-12-31 12:00:00,29.548918,53.780205
37981,37981,2021-12-31 15:00:00,40.026148,27.392646
37982,37982,2021-12-31 18:00:00,41.244712,19.955186


In [62]:
# Hacer que la 'Fecha' sea el índice
datos.set_index('Fecha', inplace=True)

#Filtra los datos desde el 2019
datos = datos[datos.index.year >= 2019]
datos

,Unnamed: 0,Temperatura,Humedad
Fecha,,,
2019-01-01 00:00:00,29216,28.260936,66.311373
2019-01-01 03:00:00,29217,27.051418,68.882454
2019-01-01 06:00:00,29218,26.234018,72.660367
2019-01-01 09:00:00,29219,25.644533,74.647767
2019-01-01 12:00:00,29220,31.437436,55.215384
...,...,...,...
2021-12-31 09:00:00,37979,21.719499,85.092387
2021-12-31 12:00:00,37980,29.548918,53.780205
2021-12-31 15:00:00,37981,40.026148,27.392646


In [63]:
datos = datos.drop(columns=['Unnamed: 0'])

In [64]:
# Eliminar filas con valores faltantes
datos.dropna(inplace=True)

In [65]:
print(datos)

                     Temperatura    Humedad
Fecha                                      
2019-01-01 00:00:00    28.260936  66.311373
2019-01-01 03:00:00    27.051418  68.882454
2019-01-01 06:00:00    26.234018  72.660367
2019-01-01 09:00:00    25.644533  74.647767
2019-01-01 12:00:00    31.437436  55.215384
...                          ...        ...
2021-12-31 09:00:00    21.719499  85.092387
2021-12-31 12:00:00    29.548918  53.780205
2021-12-31 15:00:00    40.026148  27.392646
2021-12-31 18:00:00    41.244712  19.955186
2021-12-31 21:00:00    37.645491  22.674331

[8768 rows x 2 columns]


In [66]:
datos_interpolados = datos.resample('1H').interpolate(method='spline', order=3)
print(datos_interpolados)

                     Temperatura    Humedad
Fecha                                      
2019-01-01 00:00:00    28.260936  66.311373
2019-01-01 01:00:00    28.223146  66.523607
2019-01-01 02:00:00    27.908703  67.385359
2019-01-01 03:00:00    27.051418  68.882454
2019-01-01 04:00:00    26.729742  70.296842
...                          ...        ...
2021-12-31 17:00:00    41.562780  19.987211
2021-12-31 18:00:00    41.244712  19.955186
2021-12-31 19:00:00    41.059331  21.839127
2021-12-31 20:00:00    39.683413  22.903388
2021-12-31 21:00:00    37.645491  22.674331

[26302 rows x 2 columns]


In [67]:
datos = datos_interpolados

In [68]:
# Eliminar filas con valores faltantes
datos.dropna(inplace=True)

In [69]:
# CALCULAR EL MÁXIMO DE CADA DÍA
max_temperaturas_diarias = datos_interpolados.resample('D').max()
print(max_temperaturas_diarias)

            Temperatura    Humedad
Fecha                             
2019-01-01    37.639718  74.647767
2019-01-02    38.286905  69.295977
2019-01-03    38.163652  59.960911
2019-01-04    37.351168  48.543176
2019-01-05    34.843885  92.637460
...                 ...        ...
2021-12-27    40.380080  23.540752
2021-12-28    41.448426  39.080582
2021-12-29    42.678048  25.573156
2021-12-30    43.197525  62.575756
2021-12-31    41.562780  85.092387

[1096 rows x 2 columns]


In [70]:
# Aplicar corte de cuartil a las temperaturas máximas diarias
cuartiles = max_temperaturas_diarias['Temperatura'].quantile([0.25, 0.5, 0.75])

In [71]:
# Definir categorías basadas en los cuartiles
umbral_frio = cuartiles[0.25]
umbral_bueno = cuartiles[0.50]
umbral_caliente = cuartiles[0.75]
print(cuartiles)
datos['Max_Temperatura_Dia'] = datos.groupby(datos.index.date)['Temperatura'].transform('max')
datos

0.25    27.066809
0.50    31.893574
0.75    36.407532
Name: Temperatura, dtype: float64


,Temperatura,Humedad,Max_Temperatura_Dia
Fecha,,,
2019-01-01 00:00:00,28.260936,66.311373,37.639718
2019-01-01 01:00:00,28.223146,66.523607,37.639718
2019-01-01 02:00:00,27.908703,67.385359,37.639718
2019-01-01 03:00:00,27.051418,68.882454,37.639718
2019-01-01 04:00:00,26.729742,70.296842,37.639718
...,...,...,...
2021-12-31 17:00:00,41.562780,19.987211,41.562780
2021-12-31 18:00:00,41.244712,19.955186,41.562780
2021-12-31 19:00:00,41.059331,21.839127,41.562780


In [72]:
# Asignar cada observación de temperatura a su categoría correspondiente
datos['Categoria_Temperatura'] = pd.cut(datos['Temperatura'], bins=[-float('inf'), umbral_frio, umbral_bueno, umbral_caliente, float('inf')],
                                      labels=['Frío', 'Bueno', 'Caliente', 'Muy Caliente'])

datos

,Temperatura,Humedad,Max_Temperatura_Dia,Categoria_Temperatura
Fecha,,,,
2019-01-01 00:00:00,28.260936,66.311373,37.639718,Bueno
2019-01-01 01:00:00,28.223146,66.523607,37.639718,Bueno
2019-01-01 02:00:00,27.908703,67.385359,37.639718,Bueno
2019-01-01 03:00:00,27.051418,68.882454,37.639718,Frío
2019-01-01 04:00:00,26.729742,70.296842,37.639718,Frío
...,...,...,...,...
2021-12-31 17:00:00,41.562780,19.987211,41.562780,Muy Caliente
2021-12-31 18:00:00,41.244712,19.955186,41.562780,Muy Caliente
2021-12-31 19:00:00,41.059331,21.839127,41.562780,Muy Caliente


In [73]:
# Desplazar para obtener la temperatura del siguiente día
datos['Temperatura_Siguiente_Dia'] = datos['Categoria_Temperatura'].shift(-24)
datos

,Temperatura,Humedad,Max_Temperatura_Dia,Categoria_Temperatura,Temperatura_Siguiente_Dia
Fecha,,,,,
2019-01-01 00:00:00,28.260936,66.311373,37.639718,Bueno,Caliente
2019-01-01 01:00:00,28.223146,66.523607,37.639718,Bueno,Bueno
2019-01-01 02:00:00,27.908703,67.385359,37.639718,Bueno,Bueno
2019-01-01 03:00:00,27.051418,68.882454,37.639718,Frío,Bueno
2019-01-01 04:00:00,26.729742,70.296842,37.639718,Frío,Bueno
...,...,...,...,...,...
2021-12-31 17:00:00,41.562780,19.987211,41.562780,Muy Caliente,NaN
2021-12-31 18:00:00,41.244712,19.955186,41.562780,Muy Caliente,NaN
2021-12-31 19:00:00,41.059331,21.839127,41.562780,Muy Caliente,NaN


In [94]:
# USAR ONE HOT ENCODING
datos_codificados = pd.get_dummies(datos, columns=['Categoria_Temperatura'])
datos_codificados

,Temperatura,Humedad,Max_Temperatura_Dia,Temperatura_Siguiente_Dia,Categoria_Temperatura_Frío,Categoria_Temperatura_Bueno,Categoria_Temperatura_Caliente,Categoria_Temperatura_Muy Caliente
Fecha,,,,,,,,
2019-01-01 00:00:00,28.260936,66.311373,37.639718,Caliente,False,True,False,False
2019-01-01 01:00:00,28.223146,66.523607,37.639718,Bueno,False,True,False,False
2019-01-01 02:00:00,27.908703,67.385359,37.639718,Bueno,False,True,False,False
2019-01-01 03:00:00,27.051418,68.882454,37.639718,Bueno,True,False,False,False
2019-01-01 04:00:00,26.729742,70.296842,37.639718,Bueno,True,False,False,False
...,...,...,...,...,...,...,...,...
2021-12-31 17:00:00,41.562780,19.987211,41.562780,NaN,False,False,False,True
2021-12-31 18:00:00,41.244712,19.955186,41.562780,NaN,False,False,False,True
2021-12-31 19:00:00,41.059331,21.839127,41.562780,NaN,False,False,False,True


In [102]:
# Definir características y variable objetivo
X = datos_codificados.iloc[:,0:2]# Probar diferentes combinaciones de features
y = datos_codificados.iloc[:,5] #para simplificar el problema tambien se podria hacer un modelo que prediga solo dias frios y muy calientes

In [106]:
# Normalizar características numéricas
scaler = StandardScaler()
X_escalado = scaler.fit_transform(X)

In [107]:
# Dividir datos en conjuntos de entrenamiento y validación
indices_entrenamiento = (datos.index.year <= 2019)
indices_validacion = ((datos.index.year == 2020)) #& (datos.index.year <= 2020))

print(len(indices_entrenamiento))
print(len(indices_validacion))
print(len(X))

26302
26302
26302


In [108]:
# Dividir datos en conjuntos de entrenamiento y validación
global X_entrenamiento, X_validacion
X_entrenamiento, X_validacion = X_escalado[indices_entrenamiento], X_escalado[indices_validacion]
global y_entrenamiento, y_validacion
y_entrenamiento, y_validacion = y[indices_entrenamiento], y[indices_validacion]

In [112]:
# Definir función objetivo para Optuna
def objetivo(trial):
    # Definir hiperparámetros a ajustar
    num_capas = trial.suggest_int('num_capas', 1, 3)
    tf.keras.utils.set_random_seed(SEED)

    num_unidades = trial.suggest_categorical('num_unidades', [32, 64, 128]) # modificar
    tasa_aprendizaje = trial.suggest_float('tasa_aprendizaje', 1e-4, 1e-2, log=True)

    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)

    # Definir la arquitectura del modelo
    modelo = Sequential()
    modelo.add(Dense(num_unidades, activation='relu', input_shape=(X.shape[1],)))
    for _ in range(num_capas - 1):
        modelo.add(Dense(num_unidades, activation='relu')) # se podria agregar una capa de dropout
        if dropout_rate > 0.0:
            modelo.add(tf.keras.layers.Dropout(dropout_rate))

    modelo.add(Dense(1, activation='sigmoid'))
    #modelo.add(Dense(4, activation='softmax'))  # 4 categorías: Frío, Bueno, Caliente, Muy Caliente, aqui se pueden probar otras opciones

    # Compilar el modelo
    modelo.compile(optimizer=Adam(learning_rate=tasa_aprendizaje),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])


    # Entrenar el modelo  # el batch size se puede variar tambien depediendo de la gpu que se tenga, el numero de épocas debe ser de, al menos 50 o 100
    modelo.fit(X_entrenamiento, y_entrenamiento, validation_data=(X_validacion, y_validacion), epochs=50, batch_size=128, verbose=0, shuffle=True)


    # Evaluar el modelo en el conjunto de validación
    _, val_acc = modelo.evaluate(X_validacion, y_validacion, verbose=0)

    return val_acc

# Realizar la optimización de hiperparámetros usando Optuna
estudio = optuna.create_study(direction='maximize') # cambiar si se modificar la metrica de la funcion objetivo, se puede hacer un problema multiobjetivo tambien
estudio.optimize(objetivo, n_trials=10) # probar mas trials una vez que se tenga el script funcionando, minimo 50, evitar variar muchos parametros al mismo tiempo

# Obtener los mejores hiperparámetros
mejor_num_capas = estudio.best_params['num_capas']
mejor_num_unidades = estudio.best_params['num_unidades']
mejor_tasa_aprendizaje = estudio.best_params['tasa_aprendizaje']

print("Mejores Hiperparámetros:")
print("Número de Capas:", mejor_num_capas)
print("Número de Unidades:", mejor_num_unidades)
print("Tasa de Aprendizaje:", mejor_tasa_aprendizaje)

[I 2024-05-02 18:09:55,617] A new study created in memory with name: no-name-027c7842-579d-49ec-a8ab-485edca2bc25
[I 2024-05-02 18:10:17,405] Trial 0 finished with value: 0.8727231621742249 and parameters: {'num_capas': 1, 'num_unidades': 32, 'tasa_aprendizaje': 0.0001498947478145772, 'dropout_rate': 0.08106493919317154}. Best is trial 0 with value: 0.8727231621742249.
[I 2024-05-02 18:10:39,930] Trial 1 finished with value: 0.9969262480735779 and parameters: {'num_capas': 3, 'num_unidades': 128, 'tasa_aprendizaje': 0.004489047644716372, 'dropout_rate': 0.029617024095055733}. Best is trial 1 with value: 0.9969262480735779.
[I 2024-05-02 18:10:58,377] Trial 2 finished with value: 0.9828096628189087 and parameters: {'num_capas': 1, 'num_unidades': 128, 'tasa_aprendizaje': 0.00020068338054896504, 'dropout_rate': 0.3306119557347012}. Best is trial 1 with value: 0.9969262480735779.
[I 2024-05-02 18:11:19,930] Trial 3 finished with value: 0.9968124032020569 and parameters: {'num_capas': 1, '

Mejores Hiperparámetros:
Número de Capas: 1
Número de Unidades: 32
Tasa de Aprendizaje: 0.005018107126316448


In [114]:
# Entrenar el modelo final usando los mejores hiperparámetros
modelo_final = Sequential()
modelo_final.add(Dense(mejor_num_unidades, activation='relu', input_shape=(X_entrenamiento.shape[1],)))
for _ in range(mejor_num_capas - 1):
    modelo_final.add(Dense(mejor_num_unidades, activation='relu'))
modelo_final.add(Dense(4, activation='softmax'))

modelo_final.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=mejor_tasa_aprendizaje),
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

# Dividir datos en conjuntos de entrenamiento y prueba
indices_entrenamiento_final = (datos.index.year < 2020)
indices_prueba_final = (datos.index.year >= 2021)

In [115]:
# Dividir datos en conjuntos de entrenamiento y prueba
X_entrenamiento, X_prueba = X.loc[indices_entrenamiento_final], X.loc[indices_prueba_final]
y_entrenamiento, y_prueba = y.loc[indices_entrenamiento_final], y.loc[indices_prueba_final]

In [116]:
entrenamiento = tf.keras.utils.to_categorical(y_entrenamiento, num_classes=4)
y_prueba_o= tf.keras.utils.to_categorical(y_prueba, num_classes=4)

In [117]:
# Entrenar modelo final
modelo_final.fit(X_entrenamiento, entrenamiento, validation_data=(X_prueba, y_prueba_o), epochs=50, batch_size=256, verbose=1,shuffle=False)

Epoch 1/50
35/35 [==============================] - 2s 16ms/step - loss: 1.5268 - accuracy: 0.7482 - val_loss: 0.4235 - val_accuracy: 0.7894
Epoch 2/50
35/35 [==============================] - 0s 10ms/step - loss: 0.4670 - accuracy: 0.8317 - val_loss: 0.4280 - val_accuracy: 0.8236
Epoch 3/50
35/35 [==============================] - 0s 8ms/step - loss: 0.4446 - accuracy: 0.8215 - val_loss: 0.3926 - val_accuracy: 0.8167
Epoch 4/50
35/35 [==============================] - 0s 8ms/step - loss: 0.4060 - accuracy: 0.8315 - val_loss: 0.3969 - val_accuracy: 0.8231
Epoch 5/50
35/35 [==============================] - 0s 6ms/step - loss: 0.4281 - accuracy: 0.8269 - val_loss: 0.3916 - val_accuracy: 0.8219
Epoch 6/50
35/35 [==============================] - 0s 6ms/step - loss: 0.4079 - accuracy: 0.8305 - val_loss: 0.3846 - val_accuracy: 0.8210
Epoch 7/50
35/35 [==============================] - 0s 5ms/step - loss: 0.4079 - accuracy: 0.8316 - val_loss: 0.3787 - val_accuracy: 0.8188
Epoch 8/50
35/35 [

In [122]:
#Evaluar en conjunto de prueba
_, val_acc =modelo_final.evaluate(X_prueba, y_prueba_o, verbose=0)
print(val_acc)

# Ejemplo de como calcular las otras metricas que podrias utilizarse como funcion objetivo para la optimizacion de hiperparametros en lugar del acc
y_pred = modelo_final.predict(X_prueba)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convertir las predicciones en clases

precision = precision_score(np.argmax(y_prueba_o, axis=1), y_pred_classes, average='weighted')
recall = recall_score(np.argmax(y_prueba_o, axis=1), y_pred_classes, average='weighted')
f1 = f1_score(np.argmax(y_prueba_o, axis=1), y_pred_classes, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

0.8284996747970581
274/274 [==============================] - 2s 6ms/step
Precision: 0.8252449233983064
Recall: 0.8284996574560402
F1 Score: 0.8268037357295681
